## 事件流处理器

In [1]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

EventBlock(block_type=<chunk>, content=<hi>)

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


### 推送事件流

In [3]:
for x in r(generator="sync", verbose=False):
    print("SSE >>", x)

SSE >> {'event': 'log', 'id': '20241122-89123-0016-8880', 'data': {'content': 'MyRun.4413354576', 'block_type': 'runnable', 'content_id': '20241122-89123-0017-3607', 'created_at': '2024-11-22T23:25:23.197268', 'thread_id': None, 'calling_id': '20241122-89123-0002-5685', 'agent_name': 'MyRun.4413354576', 'model_name': None}}
hi
SSE >> {'event': 'log', 'id': '20241122-89123-0020-5749', 'data': {'content': 'hi\n', 'block_type': 'text', 'content_id': '20241122-89123-0021-2579', 'created_at': '2024-11-22T23:25:23.198236', 'thread_id': None, 'calling_id': '20241122-89123-0002-5685', 'agent_name': 'MyRun.4413354576', 'model_name': None}}
illufly!
SSE >> {'event': 'log', 'id': '20241122-89123-0024-1323', 'data': {'content': 'illufly!\n', 'block_type': 'text', 'content_id': '20241122-89123-0025-0595', 'created_at': '2024-11-22T23:25:23.198424', 'thread_id': None, 'calling_id': '20241122-89123-0002-5685', 'agent_name': 'MyRun.4413354576', 'model_name': None}}


In [4]:
async for x in r(generator="async", verbose=False):
    print("SSE >>", x)

SSE >> {'event': 'log', 'id': '20241122-89126-0026-6573', 'data': {'content': 'MyRun.4413354576', 'block_type': 'runnable', 'content_id': '20241122-89126-0027-1210', 'created_at': '2024-11-22T23:25:26.138701', 'thread_id': None, 'calling_id': '20241122-89126-0003-8537', 'agent_name': 'MyRun.4413354576', 'model_name': None}}
hi
SSE >> {'event': 'log', 'id': '20241122-89126-0030-3650', 'data': {'content': 'hi\n', 'block_type': 'text', 'content_id': '20241122-89126-0031-3347', 'created_at': '2024-11-22T23:25:26.141209', 'thread_id': None, 'calling_id': '20241122-89126-0003-8537', 'agent_name': 'MyRun.4413354576', 'model_name': None}}
illufly!
SSE >> {'event': 'log', 'id': '20241122-89126-0034-2730', 'data': {'content': 'illufly!\n', 'block_type': 'text', 'content_id': '20241122-89126-0035-4262', 'created_at': '2024-11-22T23:25:26.142561', 'thread_id': None, 'calling_id': '20241122-89126-0003-8537', 'agent_name': 'MyRun.4413354576', 'model_name': None}}


In [5]:
r.events_history.store

{'20241122-89080-0000-8402': {'threads': set(),
  'callings': {'20241122-89080-0000-1432': {'id': '20241122-89080-0000-5217',
    'event_type': 'log',
    'data': {'content': 'MyRun.4413354576',
     'block_type': 'runnable',
     'content_id': '20241122-89080-0001-0418',
     'created_at': '2024-11-22T23:24:40.147866',
     'thread_id': None,
     'calling_id': '20241122-89080-0000-1432',
     'agent_name': 'MyRun.4413354576',
     'model_name': None}},
   '20241122-89080-0001-5718': {'id': '20241122-89080-0006-4167',
    'event_type': 'log',
    'data': {'content': 'MyRun.4413354576',
     'block_type': 'runnable',
     'content_id': '20241122-89080-0007-6964',
     'created_at': '2024-11-22T23:24:40.155231',
     'thread_id': None,
     'calling_id': '20241122-89080-0001-5718',
     'agent_name': 'MyRun.4413354576',
     'model_name': None}},
   '20241122-89123-0002-5685': {'id': '20241122-89123-0016-8880',
    'event_type': 'log',
    'data': {'content': 'MyRun.4413354576',
     'b

In [9]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

### Runnable 事件流

In [10]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun(handlers=[handle])
r("hi")

{"id": "c3cd3fc0a4ba11ef9c7aacde48001122", "content_id": "c3cd4042a4ba11ef9c7aacde48001122", "block_type": "runnable", "content": "MyRun.4592974432", "created_at": "2024-11-17T16:05:52.765559", "calling_info": {}, "runnable_info": {"name": "MyRun.4592974432", "class_name": "MyRun", "calling_id": "c3cd3d54-a4ba-11ef-9c7a-acde48001122"}}
{"id": "c3cd5aaaa4ba11ef9c7aacde48001122", "content_id": "c3cd5bb8a4ba11ef9c7aacde48001122", "block_type": "chunk", "content": "hi, ", "created_at": "2024-11-17T16:05:52.766262", "calling_info": {}, "runnable_info": {"calling_id": "c3cd3d54-a4ba-11ef-9c7a-acde48001122"}}
{"id": "c3cd5fb4a4ba11ef9c7aacde48001122", "content_id": "c3cd6022a4ba11ef9c7aacde48001122", "block_type": "chunk", "content": "illufly", "created_at": "2024-11-17T16:05:52.766372", "calling_info": {}, "runnable_info": {"calling_id": "c3cd3d54-a4ba-11ef-9c7a-acde48001122"}}
{"id": "c3cd628ea4ba11ef9c7aacde48001122", "content_id": "c3cd62e8a4ba11ef9c7aacde48001122", "block_type": "chunk",

### ChatAgent 事件流

In [ ]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

def handle(block, **kwargs):
    print("[", block.block_type, "]", block.text)

r = FakeLLM(handlers=[handle], sleep=0.5)
r("hi")

In [ ]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

r = FakeLLM(handlers=[alog], sleep=0.1)
await r("hi")

### 推送 ChatAgent 事件流

In [6]:
from illufly.chat import FakeLLM

llm = FakeLLM(name="if1", handlers=[])
llm.create_new_thread()

async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'event': 'log', 'id': '20241122-89246-0036-9502', 'data': {'content': 'if1', 'block_type': 'runnable', 'content_id': '20241122-89246-0037-7321', 'created_at': '2024-11-22T23:27:26.151203', 'thread_id': '20241122-89246-0000-0946', 'calling_id': '20241122-89246-0004-1536', 'agent_name': 'if1', 'model_name': None}}
{'event': 'input', 'id': '20241122-89246-0038-9467', 'data': {'content': '给我写一篇200字小作文', 'block_type': 'user', 'content_id': '20241122-89246-0039-7681', 'created_at': '2024-11-22T23:27:26.151829', 'thread_id': '20241122-89246-0000-0946', 'calling_id': '20241122-89246-0004-1536', 'agent_name': 'if1', 'model_name': None}}
{'event': 'log', 'id': '20241122-89246-0040-5620', 'data': {'content': 'if1', 'block_type': 'agent', 'content_id': '20241122-89246-0041-2518', 'created_at': '2024-11-22T23:27:26.151975', 'thread_id': '20241122-89246-0000-0946', 'calling_id': '20241122-89246-0004-1536', 'agent_name': 'if1', 'model_name': None}}
{'event': 'log', 'id': '20241122-89246-0042-2185', 

In [7]:
async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'event': 'log', 'id': '20241122-89256-0091-4574', 'data': {'content': 'if1', 'block_type': 'runnable', 'content_id': '20241122-89256-0092-7692', 'created_at': '2024-11-22T23:27:36.051562', 'thread_id': '20241122-89246-0000-0946', 'calling_id': '20241122-89256-0005-9750', 'agent_name': 'if1', 'model_name': None}}
{'event': 'input', 'id': '20241122-89256-0093-4380', 'data': {'content': '给我写一篇200字小作文', 'block_type': 'user', 'content_id': '20241122-89256-0094-7199', 'created_at': '2024-11-22T23:27:36.053071', 'thread_id': '20241122-89246-0000-0946', 'calling_id': '20241122-89256-0005-9750', 'agent_name': 'if1', 'model_name': None}}
{'event': 'log', 'id': '20241122-89256-0095-2399', 'data': {'content': 'if1', 'block_type': 'agent', 'content_id': '20241122-89256-0096-9903', 'created_at': '2024-11-22T23:27:36.053393', 'thread_id': '20241122-89246-0000-0946', 'calling_id': '20241122-89256-0005-9750', 'agent_name': 'if1', 'model_name': None}}
{'event': 'log', 'id': '20241122-89256-0097-7504', 

## 嵌套 BaseAgent

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

store = {}
flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking])
)

for b in flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False, generator="sync"):
    print(b)

{'event': 'log', 'id': '20241122-89486-0000-5272', 'data': {'content': 'ReAct.4557867328', 'block_type': 'runnable', 'content_id': '20241122-89486-0001-8567', 'created_at': '2024-11-22T23:31:26.754816', 'thread_id': None, 'calling_id': '20241122-89486-0000-8696', 'agent_name': 'ReAct.4557867328', 'model_name': None}}
{'event': 'log', 'id': '20241122-89486-0002-7206', 'data': {'content': '>>> flow-node-1: planner', 'block_type': 'node', 'content_id': '20241122-89486-0003-8546', 'created_at': '2024-11-22T23:31:26.755321', 'thread_id': None, 'calling_id': '20241122-89486-0000-8696', 'agent_name': 'ReAct.4557867328', 'model_name': None}}
{'event': 'log', 'id': '20241122-89486-0004-3324', 'data': {'content': '我在鸿蒙公司，帮我查询一下天气情况', 'block_type': 'user', 'content_id': '20241122-89486-0005-9716', 'created_at': '2024-11-22T23:31:26.755581', 'thread_id': None, 'calling_id': '20241122-89486-0000-8696', 'agent_name': 'react-planner', 'model_name': 'qwen-plus'}}
[AGENT] react-planner
{'event': 'log',

## Events History

In [14]:
from illufly.core.history.events import BaseEventsHistory

In [20]:
h = BaseEventsHistory()

In [23]:
h.store

{'20241117-28722-2213-0000': {'threads': set(), 'callings': {}}}

In [24]:
h.last_events_history_id

'20241117-28722-2213-0000'

In [25]:
24*60*60

86400